In [5]:
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

In [6]:
from dotenv import load_dotenv
import os


load_dotenv()


GROQ_API_KEY = os.getenv("GROQ_API_KEY")


if GROQ_API_KEY is None:
    raise ValueError("GROQ_API_KEY is not set. Please check your .env file.")


from langchain_groq import ChatGroq


model = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

In [7]:
model.invoke("What is the capital of France?")


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.03893, 'prompt_time': 0.01265255, 'queue_time': 0.00025875899999999903, 'total_time': 0.05158255}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-caa97d02-9789-46ba-b030-eaf9af17c7b5-0', usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50})

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("What is the capital of France?")


'The capital of France is Paris.'

In [9]:
from langchain.prompts import ChatPromptTemplate

template = """ 
Answer the following question based only on the provided context. If you cannot answer the question based on the context, say "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model | parser

print(chain)

chain.invoke({
    "context": "The capital of France is Paris.", 
    "question": "What is the capital of France?"
    })


first=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' \nAnswer the following question based only on the provided context. If you cannot answer the question based on the context, say "I don\'t know".\n\nContext: {context}\n\nQuestion: {question}\n'), additional_kwargs={})]) middle=[ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D183C9D040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D183C9DA00>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))] last=StrOutputParser()


'The capital of France is Paris.'

In [10]:
translation_prompt = ChatPromptTemplate.from_template(
    """
    Translate {answer} to {language}
    """
)

from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke({
    "context": "The capital of France is Paris.", 
    "question": "What is the capital of France?",
    "language": "French"
    })



'La capitale de la France est Paris.'

In [22]:
import os
import tempfile
import whisper
from pytubefix import YouTube

if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()
    file = audio.download()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w+") as file:
            file.write(transcription)
            

FileNotFoundError: [WinError 2] The system cannot find the file specified